In [116]:
import csv
import matplotlib
import pandas as pd
import numpy as np
import datetime
import random
import matplotlib.pyplot as plt
import os

import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.offline as py
import plotly.graph_objs as go

from ggplot import *

# For Live Experiment

In [117]:

def parse(ip, data):

    x = np.array(data[data['host_ip'] == ip]['test_date'])
    y = np.array(data[data['host_ip'] == ip]['discrimination_losses_str'])
    z = np.array(data[data['host_ip'] == ip]['base_losses_str'])

    x_ax = []
    y_ax = []
    z_ax = []
    diff = []
    for i in range(0, len(x)):
        #x_ax.append(str(x[i][:][6:16]))
        if i <= len(x) - 13:
            x_ax.append(i+1)
        if (i >= 5 and i < 11) or i >= 19:
            y_ax.append(str((y[i]/5000) * 100))
            z_ax.append(str((z[i]/5000) * 100))
            diff.append( str(abs(((y[i]/5000) * 100) - ((z[i]/5000) * 100))))
        
    return x_ax, y_ax, z_ax, diff

In [118]:
def draw(x_axis, y_axis, z_axis, diff, t, place ,isSecondDay):
    
    threshold  = []
    for i in range(50):
        threshold.append(20)
        
    # use below code to generate graphs with markrs only, or markers+line
    trace1 = {"x":x_axis, "y":y_axis, "marker": {"color": "#CD5C5C", "size": 6, "symbol":"x"},
              "name": "Discriminated",
              "type": "scatter",
              "mode": "lines+markers",
              "line":{"width":1}
              }

    trace2 = {"x":x_axis, "y":z_axis, "marker": {"color": "#3cb371", "size": 6, "symbol":"circle"},
              "name": "Base",
              "type": "scatter",
              "mode": "lines+markers",
              "line":{"width":1}
             }


    trace3 = {"x":x_axis, "y":diff, "marker": {"color": "#87cefa", "size": 6, "symbol":"triangle-up"},
              "name": chr(9651),
              "type": "scatter",
              "mode": "lines+markers", #lines+markers
              "line":{"width":1}
             }
    
    
    # uncomment below code to generate dash lines only
    
#     trace1 = go.Scatter(
#         x = x_axis,
#         y = y_axis,
#         marker={"symbol":"x"},
#         name = 'Discriminated', 
#         line = dict(
#             color = ('#CD5C5C'),
#             width = 1,
#             dash = 'dash')
#     )

#     trace2 = go.Scatter(
#         x = x_axis,
#         y = z_axis,
#         marker={"symbol":"circle"}, 
#         name = 'Base', 
#         line = dict(
#             color = ('#3cb371'),
#             width = 1,
#             dash = 'dash')
#     )

#     trace3 = go.Scatter(
#         x = x_axis,
#         y = diff,
#         marker={"symbol":"triangle-up"}, 
#         name = chr(9651), 
#         line = dict(
#             color = ('#87cefa'),
#             width = 1,
#             dash = 'dash')
#     )

    
    #Trace 4 is only used for 20Percent threshold
    trace4 = go.Scatter(
        x = x_axis,
        y = threshold,
        showlegend=False,
        line = dict(
            color = ('grey'),
            width = 1,
            dash = 'dash')
    )


    data = [trace1, trace2, trace3, trace4]

    
    layout = go.Layout(
        width = 2500,   # width and height of the image it self
        height = 2500 ,
        legend=dict(
            x=1.01,        # x position of legend
            y=.5,          # y position of legend
            font=dict(
                size=18   # Size of legend
            )        
        ),
        
        # Cosmetics for X axis
        xaxis=dict(
            title = "Experiment ID",    #title
            showgrid=True,              # turn grid on or off
            showticklabels=True,
            #tickangle=45,  # Angle
            showline=True,
            mirror='ticks',
            linewidth=1,       # size of border, up and down
            range=[0,40.5], 
            titlefont=dict(
                size=21           # size of xaxis title
            )            
            
        ),
        yaxis=dict(
            title="Loss Rate (%)",
            range=[0,102],
            showgrid=True,
            showline=True,
            mirror='ticks',
            linewidth=1,      #size of border, left and right
            titlefont=dict(
                size=21        #size of yaxis title
            )               
        ),
        
        annotations=[
            dict(
                x=10,            #x position of 20percent threshold writing #30
                y=21.5,         # y position of "   ""       ""     """"
                xref='x',
                yref='y',
                text='20% Threshold',    # The writing
                showarrow=False,
            )
        ]        

        #title = t + ' (' + place + ')',           #graph title
    )

    fig = go.Figure(data=data, layout=layout)

    py.iplot(fig, filename = t ,image='svg')

In [119]:
# Insert the name of csv file inside read_csv function to produce graphs
data = pd.read_csv("CompressionRetry1000.csv")

#Source Ip is a dictionary and Country is a list of coresponding IPS and Countries
sourceIp = {'131.179.150.72':'planetlab2.cs.ucla.edu', '192.16.125.12':'planetlab-2.ssvl.kth.se', '165.242.90.129':'pl2.sos.info.hiroshima-cu.ac.jp', '192.91.235.230':'pluto.cs.brown.edu'}
country = ['UCLA','Sweden','Japan','Brown Univeristy']


count = 0
for key in sourceIp.keys():
    x ,y, z, diff = parse(key, data)
    draw(x,y,z,diff,sourceIp[key], country[count] ,isSecondDay = False)
    count = count + 1

# Run for Experiment Without lines

In [120]:
def drawSim(file, isCompression):
    
    df = pd.read_csv(file)
    arr = []
    columnNames = list(df.head(0))
    axisTitle =''
    
    if columnNames[0] == 'Bottleneck':
        axisTitle = 'Bottleneck'
        for i in df[columnNames[0]]:
            arr.append(i[:-4])
    else:
        axisTitle = 'Number of Packets'
    
    if isCompression:

        trace1 = {"x":df[columnNames[0]], "y":df[columnNames[2]], "marker": {"color": "red", "size": 8, "symbol":"diamond"},
                  "mode": "markers",
                  "name": "Discriminated",
                  "type": "scatter",
                  "mode":"markers"
                  }

        trace2 = {"x":df[columnNames[0]], "y":df[columnNames[1]], "marker": {"color": "green", "size": 8, "symbol":"square"},
                  "mode": "markers",
                  "name": "Base",
                  "type": "scatter",
                  "mode":"markers"
                 }


        trace3 = {"x":df[columnNames[0]], "y":df[columnNames[3]], "marker": {"color": "blue", "size": 8},
                  "mode": "markers",
                  "name": chr(9651),
                  "type": "scatter",
                  "mode":"markers"
                 }    
        
    else:
        
        trace1 = {"x":df[columnNames[0]], "y":df[columnNames[1]], "marker": {"color": "red", "size": 8, "symbol":"x"},
                  "mode": "markers",
                  "name": chr(9651) + columnNames[1],   #"Base",
                  "type": "scatter",
                  "mode":"markers"
                  }

        trace2 = {"x":df[columnNames[0]], "y":df[columnNames[2]], "marker": {"color": "green", "size": 8, "symbol":"triangle-up-open"},
                  "mode": "markers",
                  "name": "name": chr(9651) + columnNames[2], #"Discriminated",
                  "type": "scatter",
                  "mode":"markers"
                 }


        trace3 = {"x":df[columnNames[0]], "y":df[columnNames[3]], "marker": {"color": "blue", "size": 8,"symbol":"circle-open"},
                  "mode": "markers",
                  "name": "name": chr(9651) + columnNames[3], #chr(9651),
                  "type": "scatter",
                  "mode":"markers"
                 }
        trace4 = {"x":df[columnNames[0]], "y":df[columnNames[4]], "marker": {"color": "black", "size": 8, "symbol":"triangle-up"},
                  "mode": "markers",
                  "name": "name": chr(9651) + columnNames[4],,
                  "type": "scatter",
                  "mode":"markers"
                 }


        trace5 = {"x":df[columnNames[0]], "y":df[columnNames[5]], "marker": {"color": "yellow", "size": 8,"symbol":"square"},
                  "mode": "markers",
                  "name": chr(9651),
                  "type": "scatter",
                  "mode":"markers"
                 }           

    data = [trace1,trace2,trace3,trace4,trace5]


    layout = go.Layout(
        width = 2500,   # width and height of the image it self
        height = 2500 ,
        legend=dict(
            x=1.01,        # x position of legend
            y=.5,          # y position of legend
            font=dict(
                size=18   # Size of legend
            )        
        ),
        
        # Cosmetics for X axis
        xaxis=dict(
            title = axisTitle,    #title
            showgrid=True,              # turn grid on or off
            showticklabels=True,
            #tickangle=45,  # Angle
            showline=True,
            mirror='ticks',
            linewidth=1,       # size of border, up and down
            #range=[0,10.5], 
            titlefont=dict(
                size=21           # size of xaxis title
            )            
            
        ),
        yaxis=dict(
            title="Loss Rate (%)",
            range=[0,102],
            showgrid=True,
            showline=True,
            mirror='ticks',
            linewidth=1,      #size of border, left and right
            titlefont=dict(
                size=21        #size of yaxis title
            )               
        ),     

        #title = t + ' (' + place + ')',           #graph title
    )

    fig = go.Figure(data=data, layout=layout)

    py.iplot(fig, filename = file ,image='svg')



SyntaxError: invalid syntax (<ipython-input-120-620b00013f69>, line 50)

In [ ]:
for root, dirs, filenames in os.walk("Simulation/"):
    for file in filenames:
        drawSim('Simulation/'+file,True)


# Run for Simulation With Lines

In [ ]:
def drawSimDash(file, isCompression):
    
    df = pd.read_csv(file)
    arr = []
    columnNames = list(df.head(0))
    
    if columnNames[0] == 'Bottleneck':
        for i in df[columnNames[0]]:
            arr.append(i[:-4])
            
    
    if isCompression:

        trace1 = go.Scatter(
        x = df[columnNames[0]],
        #x=arr,
        y = df[columnNames[2]],
        name = 'Discriminated', 
        line = dict(
            color = ('red'),
            width = 4,
            dash = 'dash')
        )
            
        trace2 = go.Scatter(
        x = df[columnNames[0]],
        #x=arr,
        y = df[columnNames[1]],
        name = 'Base', 
        line = dict(
            color = ('green'),
            width = 4,
            dash = 'dash')    
        )
        
        trace3 = go.Scatter(
        x = df[columnNames[0]],
        #x=arr,
        y = df[columnNames[3]],
        name = chr(9651), 
        line = dict(
            color = ('blue'),
            width = 4,
            dash = 'dash')      
        )


        
    else:
            
        trace1 = go.Scatter(
        #x = df[columnNames[0]],
        x = arr,
        y = df[columnNames[1]],
        marker={"symbol":"x"},     
        name = 'Base', 
        line = dict(
            color = ('green'),
            width = 4,
            dash = 'dash')
        )
            
        trace2 = go.Scatter(
        #x = df[columnNames[0]],
        x = arr,    
        y = df[columnNames[2]],
        marker={"symbol":"circle-open"}, 
        name = 'Discriminated', 
        line = dict(
            color = ('red'),
            width = 4,
            dash = 'dash')   
        )
        
        trace3 = go.Scatter(
        x = arr,    
        #x = df[columnNames[0]],
        y = df[columnNames[3]],
        marker={"symbol":"triangle-up-open"},        
        name = chr(9651),
        line = dict(
            color = ('blue'),
            width = 4,
            dash = 'dash')   
        )

    data = [trace1,trace2]


    layout = go.Layout(
        width = 2500,   # width and height of the image it self
        height = 2500 ,
        legend=dict(
            x=1.01,        # x position of legend
            y=.5,          # y position of legend
            font=dict(
                size=18   # Size of legend
            )        
        ),
        
        # Cosmetics for X axis
        xaxis=dict(
            title = "Bottleneck (Mbps)",    #title
            showgrid=True,              # turn grid on or off
            showticklabels=True,
            #tickangle=45,  # Angle
            showline=True,
            mirror='ticks',
            linewidth=1,       # size of border, up and down
            #range=[0,32.5], 
            titlefont=dict(
                size=21           # size of xaxis title
            )            
            
        ),
        yaxis=dict(
            title="Loss Rate (%)",
            range=[0,102],
            showgrid=True,
            showline=True,
            mirror='ticks',
            linewidth=1,      #size of border, left and right
            titlefont=dict(
                size=21        #size of yaxis title
            )               
        ),     

        #title = t + ' (' + place + ')',           #graph title
    )


    fig = go.Figure(data=data, layout=layout)

    py.iplot(fig, filename = file ,image='svg')



In [ ]:
for root, dirs, filenames in os.walk("Simulation/SPQNEW"):
    for file in filenames:
        drawSimDash('Simulation/SPQNEW/'+file,False)

# df = pd.read_csv('Simulation/SPQNEW/spq_loss_rate_ratio_n_prime_2.csv')
# columnNames = list(df.head(0))
# print(columnNames)


# Delayer Graphs

In [ ]:
def drawdelayer(file):
    
    df = pd.read_csv(file)        
    columnNames = list(df.head(0))

#     trace1 = go.Scatter(
#     x = df[columnNames[0]],
#     y = df[columnNames[2]],
#     name = 'Discriminated', 
#     marker={"symbol":"x"},
#     line = dict(
#         color = ('red'),
#         width = 4,
#         dash = 'dash')
#     )

#     trace2 = go.Scatter(
#     x = df[columnNames[0]],
#     y = df[columnNames[1]],
#     name = 'Base', 
#     marker={"symbol":"circle"},    
#     line = dict(
#         color = ('green'),
#         width = 4,
#         dash = 'dash')    
#     )

#     if not isDelayer:
#         trace3 = go.Scatter( 
#         x = df[columnNames[0]],
#         y = df[columnNames[3]],
#         marker={"symbol":"triangle-up"},        
#         name = chr(9651),
#         line = dict(
#             color = ('blue'),
#             width = 4,
#             dash = 'dash')   
#         )        

    trace1 = {"x":df[columnNames[0]], "y":df[columnNames[2]], "marker": {"color": "red", "size": 8, "symbol":"x"},
              "mode": "markers",
              "name": str(columnNames[2]),
              "type": "scatter",
              "mode":"markers"
              }

    trace2 = {"x":df[columnNames[0]], "y":df[columnNames[1]], "marker": {"color": "green", "size": 8, "symbol":"triangle-up"},
              "mode": "markers",
              "name": str(columnNames[1]),
              "type": "scatter",
              "mode":"markers"
             }
    
    trace3 = {"x":df[columnNames[0]], "y":df[columnNames[3]], "marker": {"color": "blue", "size": 8, "symbol":"diamond"},
              "mode": "markers",
              "name": str(columnNames[3]),
              "type": "scatter",
              "mode":"markers"
             }
    trace4 = {"x":df[columnNames[0]], "y":df[columnNames[4]], "marker": {"color": "yello", "size": 8, "symbol":"circle"},
              "mode": "markers",
              "name": str(columnNames[4]),
              "type": "scatter",
              "mode":"markers"
             }    


#     trace3 = {"x":df[columnNames[0]], "y":df[columnNames[0]], "marker": {"color": "blue", "size": 8},
#               "mode": "markers",
#               "name": chr(9651),
#               "type": "scatter",
#               "mode":"markers"
#              }  
        
    data = [trace1,trace2,trace3,trace4]


    layout = go.Layout(
        width = 2500,   # width and height of the image it self
        height = 2500 ,
        legend=dict(
            x=1.01,        # x position of legend
            y=.5,          # y position of legend
            font=dict(
                size=18   # Size of legend
            )        
        ),
        
        # Cosmetics for X axis
        xaxis=dict(
            title = "Number of Packets",    #title
            showgrid=True,              # turn grid on or off
            showticklabels=True,
            #tickangle=45,  # Angle
            showline=True,
            mirror='ticks',
            linewidth=1,       # size of border, up and down
            range=[0.5,5001.5], 
            titlefont=dict(
                size=21           # size of xaxis title
            )            
            
        ),
        yaxis=dict(
            title="Loss Rate (%)",
            range=[0,102],
            showgrid=True,
            showline=True,
            mirror='ticks',
            linewidth=1,      #size of border, left and right
            titlefont=dict(
                size=21        #size of yaxis title
            )               
        ),     

        #title = t + ' (' + place + ')',           #graph title
    )


    fig = go.Figure(data=data, layout=layout)

    py.iplot(fig, filename = file, image='svg')



In [ ]:
for root, dirs, filenames in os.walk("Simulation/delayer"):
    for file in filenames:
        drawdelayer("Simulation/delayer/"+file)
   
        

In [ ]:
df = pd.read_csv("Simulation/shaper/shaper__bottleneck_loss_rate_ratio_with_ni_sp_3.csv") 
columnNames = list(df.head(0))
print(columnNames)